# Learning PySpark

**Status:** *ongoing*

---

## Summary

1. What is Apache Spark?
2. Architecture
3. Spark Ecosystem
4. Resilient Distributed Dataset (RDD)
5. Introduction to SparkSQL and DataFrames: 
    + Reading & Writing Data
    + The DataFrames/Datasets API
    + Spark SQL
    + Caching and caching storage levels

---

## 1. What is Apache Spark?

[Spark](http://spark.apache.org/) is a general-purpose, distributed programming framework that was developed at the AMPLab at the University of California, Berkeley. It is open source software that provides an in-memory computation framework and it is also good for batch processing. Spark works well with real-time (or, better to say, near-real-time) data. It allows you to solve a wide variety of complex data problems whether semi-structured, structured, streaming, and/or machine learning / data sciences.

Machine learning and graph algorithms are iterative. Where Spark do magic. According to its research paper, it is approximately 100 times faster than its peer, Hadoop. Data can be cached in memory. Caching intermediate data in iterative algorithms provides amazingly fast processing speed. Spark can be programmed with Java, Scala, Python, and R. Also, Spark supports multiple data sources such as Parquet, JSON, Hive, Cassandra, CSV, text files and RDBMS tables.

Spark might be considered as an improved [Hadoop](https://hadoop.apache.org/). Because we can implement a [MapReduce](https://hadoop.apache.org/docs/r1.2.1/mapred_tutorial.html) algorithm in Spark, Spark uses the benefit of HDFS; this means Spark can read data from HDFS and store data to HDFS too, and Spark handles iterative computation efficiently because data can be persisted in memory. It is good for interactive data analysis.

#### Advantages of Spark
1. **Swift Processing:** Spark reduce the number of read-write to disk.

2. **Dynamic in Nature:** Spark provide 80 high-level operators, which can help to develop a parallel application. For transformations, Spark adds them to a DAG (Directed Acyclic Graph) of computation and only when the driver requests some data, does this DAG actually gets executed.

3. **In-Memory Computation:** We didn’t waste our time to fetch data from disk every time, it saves time by caching data.

4. **Re-Usability:** The Spark code can be reused for batch-processing, join stream against historical data.

5. **Fault Tolerance:** Through RDD, it provides fault tolerance. Spark RDD are designed to handle the failure of any worker node in the cluster, which ensures that the loss of data is reduced to zero.


<center> Logistic Regression </center>
![logistic-regression](https://user-images.githubusercontent.com/9319823/46016970-9fb87380-c0d6-11e8-86e0-7123a95c0309.png)


#### Disadvantages of Spark
1. **Expensive:** In-memory capability can become a bottleneck when we want cost-efficient processing of big data as keeping data in memory is quite expensive.

2. **Latency:** Apache Spark has higher latency as compared to [Apache Flink](https://flink.apache.org/).

3. **Mannual Optimization:** The Spark job requires to be manually optimized and is adequate to specific datasets.

4. **No File Management:** Apache Spark does not have its own file management system, thus it relies on some other platform like Hadoop.

5. **Problem With Small Files:** If we use Spark with Hadoop, we come across a problem of a small file. HDFS provides a limited number of large files rather than a large number of small files. Another place where Spark legs behind is we store the data gzipped in S3.

## 2. Architecture

![spark_architecture](https://user-images.githubusercontent.com/9319823/45994904-09645d80-c096-11e8-87e4-2b53f058ba99.png)

The main components of the Spark architecture are the driver and executors. For each PySpark application, there will be one driver program and one or more executors running on the cluster slave machine. Therefore, Spark follows a master/slave architecture.

- **Driver process / Master (Master Daemon):** The driver is the process that coordinates with many executors running on various slave machines. Spark follows a master/slave architecture.

    - The ***SparkContext*** object is created by the driver, and it is the main entry point to a (Py)Spark application.
    - Spark Driver also contains various components such as *DAGScheduler*, *TaskScheduler*, *BackendScheduler* and *BlockManager* which is responsible for the translation of spark user code into actual spark jobs executed on the cluster. 


- **Executors / Slaves (Worker Daemon):** Executors are slave processes. An executor runs tasks. It also has the capability to cache data in memory.

    - Executor is a distributed agent responsible for the execution of tasks. Every spark applications has its own executor process.
    - They usually run for the entire lifetime of a Spark application and this phenomenon is known as **“Static Allocation of Executors”**.
    - However, users can also opt for dynamic allocations of executors wherein they can add or remove spark executors dynamically to match with the overall workload.
    - Executor performs all the data processing.
    - Reads from and Writes data to external sources.
    - Executor stores the computation results data in-memory, cache or on hard disk drives.
    - Interacts with the storage systems.
    
    
- **Cluster Manager:** An external service responsible for acquiring resources on the spark cluster and allocating them to a spark job.
    - Hadoop YARN, Apache Mesos are examples of cluster manager.
    - Locally - simple standalone spark cluster manager.
    - Choosing a cluster manager for any spark application depends on the goals of the application because all cluster managers provide different set of scheduling capabilities.

The driver breaks our application into small tasks; a task is the smallest unit of your application. Tasks are run on different executors in parallel, one task per partition. The driver is also responsible for scheduling tasks to different executors. Also, The **cluster manager** manages cluster resources. The driver talks to the cluster manager to negotiate resources. The cluster manager also schedules tasks on behalf of the driver on various slave executor processes.

Spark is dispatched with Standalone Cluster Manager. However, it can also be configured on [YARN](https://hadoop.apache.org/docs/current/hadoop-yarn/hadoop-yarn-site/YARN.html) and [Apache Mesos](http://mesos.apache.org/). Spark can be also started in local mode too (i.e. on a single machine).

## 3. Spark Ecosystem

<br>

![spark-stack](https://user-images.githubusercontent.com/9319823/45998657-ca3d0900-c0a3-11e8-8bb8-32672e87d119.png)

<br>

Spark ecosytem has six components: [Spark core](https://spark.apache.org/docs/1.6.0/index.html), [SQL and DataFrames](http://spark.apache.org/sql/), [MLlib](http://spark.apache.org/mllib/) for machine learning, [GraphX](http://spark.apache.org/graphx/), [Spark Streaming](http://spark.apache.org/streaming/), and [SparkR](https://spark.apache.org/docs/1.6.0/sparkr.html). You can combine these libraries seamlessly in the same application.

#### **Spark Core:** 

All the functionalities being provided by Apache Spark are built on the top of Spark Core. It delivers speed by providing in-memory computation capability. Thus Spark Core is the foundation of parallel and distributed processing of huge dataset.

Spark Core is embedded with a special collection called **RDD** (resilient distributed dataset). RDD is among the abstractions of Spark. **Spark RDD handles partitioning data across all the nodes in a cluster**. It holds them in the memory pool of the cluster as a single unit. There are two operations performed on RDDs: Transformation and Action.
   - **Transformation:** It is a function that produces new RDD from the existing RDDs.
   - **Action:** In Transformation, RDDs are created from each other. But when we want to work with the actual dataset, then, at that point we use Action.

They are **evaluated lazily** (i.e, the execution will not start until an action is triggered which increases manageability, saves computation and thus increases optimization & speed) and the **transformations are stored as directed acyclic graphs (DAG)**. So, every action on the RDD will make Apache Spark recompute the DAG.

Apache Spark supports two types of partitioning **Hash Partitioning** and **Range Partitioning**. Available Resources, External Data Sources and Transformations used to derive RDD are correlated to the choice of partitioning technique.

Basics of **partitioning**:
- Every node in a Spark cluster contains one or more partitions;
- Partitions in Spark do not span multiple machines;
- Tuples in the same partition are guaranteed to be on the same machine;
- Spark assigns one task per partition and each worker can process one;
- The number of partitions used in Spark is configurable and having too few (causing less concurrency, data skewing & improper resource utilization) or too many (causing task scheduling to take more time than actual execution time) partitions is not good. By default, it is set to the total number of cores on all the executor nodes.

**Key features of Spark Core are:** essential I/O functionalities, task dispatching, fault recovery, and significant in programming and observing the role of the Spark cluster.

![pg57h](https://user-images.githubusercontent.com/9319823/46287815-10063f80-c584-11e8-9f03-88f5d7d033d5.png)

#### **SparkSQL:** 

SparkSQL library is a wrapper over the PySpark core that applies SQL-like analysis on a huge amount of structured or semistructured data. We can also use SQL queries with PySparkSQL. We can connect it to Apache Hive, and HiveQL can be applied too. PySparkSQL introduced the DataFrame, which is a tabular representation of structured data that is like a table in a relational database management system.

It is a distributed framework for structured data processing. Using Spark SQL, **Spark gets more information about the structure of data and the computation**. With this information, Spark can perform extra optimization. It uses same execution engine while computing an output. It **does not depend on API/ language to express the computation**.

It also enables powerful, interactive, analytical application across both streaming and historical data. Spark SQL is Spark module for structured data processing. Thus, it acts as a distributed SQL query engine.

**Key features of SparkSQL include:** cost based optimizer, mid query fault-tolerance, full compatibility with existing Hive data, etc.


#### **SparkML:** 

MLlib is a wrapper over the PySpark core that deals with machine-learning algorithms. The machine-learning API provided by the MLlib library is easy to use. MLlib supports many machine-learning algorithms for classification, clustering, text analysis, and more. Also, some lower level machine learning primitives like generic gradient descent optimization algorithm are also present in MLlib.

In Spark Version 2.0, the DataFrame-based API is the primary Machine Learning API for Spark. So, from now MLlib will not add any new feature to the RDD based API. The reason behind this is that it is more user-friendly than RDD. Some of the benefits of using DataFrames are it includes Spark Data sources, SQL DataFrame queries Tungsten and Catalyst optimizations, and uniform APIs across languages. MLlib also uses the **linear algebra package Breeze**. Breeze is a collection of libraries for numerical computing and machine learning.


#### **GraphX:** 

GraphX is Apache Spark's API for graphs and graph-parallel computation. It unifies ETL, exploratory analysis, and iterative graph computation within a single system. You can view the same data as both graphs and collections, transform and join graphs with RDDs efficiently, and write custom iterative graph algorithms using the Pregel API.

**Clustering, classification, traversal, searching, and pathfinding** is also possible in graphs. Furthermore, GraphX extends Spark RDD by bringing in light a new Graph abstraction: a directed multigraph with properties attached to each vertex and edge. GraphX also optimizes the way in which we can represent vertex and edges when they are primitive data types. To support graph computation it supports fundamental operators (e.g., subgraph, join Vertices, and aggregate Messages) as well as an optimized variant of the Pregel API.


#### **Spark Streaming:** 

Spark Streaming brings Apache Spark's language-integrated API to stream processing, letting you write streaming jobs the same way you write batch jobs. It supports Java, Scala and Python. It is an extension of the core Spark API that enables scalable, high-throughput, fault-tolerant stream processing of live data streams.

Spark can access data from sources like Kafka, Flume, Kinesis or TCP socket. The data so received is given to file system, databases and live dashboards. Spark uses Micro-batching for real-time streaming. **Micro-batching** is a technique that allows a process or task to treat a stream as a sequence of small batches of data. Hence Spark Streaming, groups the live data into small batches. 

Spark Streaming works in three phases: **(1) gathering**, **(2) processing**, and **(3) data storage**.
  1. It provides two categories of built-in streaming sources: 
      - **Basic sources:** file systems and socket connections; and 
      - **Advanced sources:** sources like Kafka, Flume, Kinesis, etc.;
  2. The gathered data is processed using complex algorithms expressed with a high-level function.
  3. The Processed data is pushed out to file systems, databases, and live dashboards.

**DStream** in Spark signifies continuous stream of data. We can form DStream in two ways either from sources such as Kafka, Flume, and Kinesis or by high-level operations on other DStreams. Thus, DStream is internally a sequence of RDDs.

![streaming-arch](https://user-images.githubusercontent.com/9319823/45999822-150c5000-c0a7-11e8-8a8a-f88b2c5b1c88.png)


#### **SparkR:** 

The key component of SparkR is SparkR DataFrame. DataFrames are a fundamental data structure for data processing in R. The concept of DataFrames extends to other languages with libraries like Pandas etc.

**R also provides software facilities for data manipulation, calculation, and graphical display**. Hence, the main idea behind SparkR was to explore different techniques to integrate the usability of R with the scalability of Spark.

## 4. RDD (Resilient Distributed Dataset)

A Resilient Distributed Datasets (RDD), the basic abstraction in Spark. Represents an **immutable, partitioned collection of elements** that can be operated on in parallel. This class contains the basic operations available on all RDDs, such as *map*, *filter*, and *persist*.

RDDs are a collection of various data that are so big in size, that they cannot fit into a single node and should be partitioned across various nodes. As a quick reminder from the last section, Apache Spark automatically partitions RDDs and distributes the partitions across different nodes.


**Creating RDD**

In [6]:
# 1 - PARALLELIZED COLLECTION
temperature = sc.parallelize([28.9, 30.6, 25.0, 29.1, 32.3, 31.0])

# 2 - EXTERNAL DATASETS
myText = sc.textFile("data/simple_text.txt")

# 3 - FROM ANOTHER RDD
high_temps = temperature.filter(lambda t: t >= 30)

***
**Notes:**

(1) The key point to note in parallelized collection is the number of partition the dataset is cut into. Spark will run **one task for each partition of cluster**. We require **two to four partitions for each CPU in cluster**. Spark sets number of partition based on our cluster.

(2) It is possible to read files in different file systems such as local file system, HDFS, Cassandra, HBase, etc. Formats such as CSV and JSON can also be load into a RDD.
***

**Returning RDD data**

In [7]:
myText.collect()

['Apache Spark is a fast and general-purpose cluster computing system. It provides high-level APIs in Java, Scala, Python and R, and an optimized engine that supports general execution graphs. It also supports a rich set of higher-level tools including Spark SQL for SQL and structured data processing, MLlib for machine learning, GraphX for graph processing, and Spark Streaming.']

In [8]:
temperature.take(2)

[28.9, 30.6]

***
**Notes:** 

(1) The action operation **collect()** should be used carefully since it returns all data. In this example, it returns the whole text as a list. However, you shoul not use it in large datasets. 

(2) The action operation **take(n)** returns *n* values/results. If you want to seet all results, use **collect()**.
***

**Filtering data**

In [9]:
range_temp = temperature.filter(lambda t: (t > 20) & (t < 29))
range_temp.collect()

[28.9, 25.0]

***
**Notes:** 

(1) **filter()** returns a new RDD, containing only the elements that meet a predicate.
***

**Example of operations on RDD 01** Word count and save output into a file.**

In [10]:
wc = myText.flatMap(lambda x: x.split(" ")).map(lambda a: (a,1)).reduceByKey(lambda a,b: a + b)
wc.saveAsTextFile("output/simple_word_count")

***
**Notes:**

(1) **flatMap()** takes a line from input RDD and apply a function on that line, and returns a list of elements.

(2) **map()** takes a line from input RDD and apply a function on that line, and returns only one element.

(3) **reduceByKey()**, in a dataset (K, V), the pairs on the same machine with the same key are combined, before the data is shuffled.

(4) **saveAsTextFile()** saves the RDD object as a text file, using string representations of elements.
***

## 5. Introduction to SparkSQL and DataFrames

**Spark SQL is a Spark module for structured data processing**. It originated as Apache Hive to run on top of Spark and is now integrated with the Spark stack. Howerver, Apache Hive had certain limitations such as no resume capability and bad performance MapReduce in medium-to-big sized datasets. Spark SQL was built to overcome these drawbacks and replace Apache Hive.

Unlike the basic Spark RDD API, the interfaces provided by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally, Spark SQL uses this extra information to perform extra optimizations.

It is organized into four libraries as follows:

<center>Libraries</center>                       | <center>Description</center>
---------------------------------------------- | ------------------------------------------------------
<center>Data Source API</center>               | This is a universal API for loading and storing structured data.
<center>DataFrame API</center>                 | It is a distributed collection of data organized into named column
<center>SQL Interpreter And Optimizer</center> | It is based on functional programming constructed in Scala
<center>SQL Service</center>                   | SQL Service is the entry point for working along structured data in Spark


Important classes of Spark SQL (**pyspark.sql.*(class)***) and DataFrames are described in the following table:


<center>Classes</center>                | <center>Description</center>
--------------------------------------- | ------------------------------------------------------
<center>SparkSession</center>           | Main entry point for **DataFrame** and SQL functionality
<center>Dataframe</center>              | A distributed collection of data grouped into named columns
<center>Column</center>                 | A column expression in a **DataFrame**
<center>Row</center>                    | A row of data in a DataFrame
<center>GroupedData</center>            | Aggregation methods, returned by **DataFrame.groupBy()**
<center>DataFrameNaFunctions</center>   | Methods for handling missing data (null values)
<center>DataFrameStatFunctions</center> | Methods for statistics functionality
<center>Functions</center>              | List of built-in functions available for **DataFrame**
<center>types</center>                  | List of data types available
<center>Window</center>                 | For working with window functions

<br><br>

One use of Spark SQL is to execute SQL queries. Spark SQL can also be used to read data from an existing Hive installation. Spark SQL supports the HiveQL syntax as well as **Hive SerDes** and **UDFs**, allowing you to access existing Hive warehouses.  <br><br>

![sql-hive-arch](https://user-images.githubusercontent.com/9319823/47175460-71b00300-d313-11e8-9a09-5e8f3bdb97dc.png)

<br>

<center>Lirbaries</center>              | <center>Description</center>
--------------------------------------- | ------------------------------------------------------
<center>Metastore</center>              | Stores metadata for each of the tables such as their schema and location
<center>HiveQL</center>                 | Hive Query Language based on SQL, but it does not strictly follow the full *SQL-92* standard
<center>UDFs</center>                   | It defines new Column-based functions that extend the vocabulary of Spark SQL’s DSL for transforming Datasets. UDFs are black boxes in their execution
<center>SerDes</center>                 | The interface handles both serialization and deserialization (I/O) and also interpreting the results of serialization as individual fields for processing




### Creating a spark instance

In [11]:
spark = SparkSession.builder \
                    .appName("learnig_pyspark") \
                    .config(conf=SparkConf()).getOrCreate()

### Reading & Writing Data

Reading from CSV file

In [20]:
myData_csv = spark.read.csv("/usr/local/spark/examples/src/main/resources/people.csv", header="true", sep=";")

Reading from JSON file

In [17]:
myData_json = spark.read.json("/usr/local/spark/examples/src/main/resources/people.json")

Saving into a file

In [29]:
# 1st option
myData_csv.write.format("parquet").save("output/people_parquet")

# 2nd option
# myData_csv.write.save("output/people_parquet", format="parquet", mode="overwrite")

| Save mode                | Description |
| :---:                    | :---        |
| error                    | (Default) When saving a DataFrame to a data source, if data already exists, an exception is expected to be thrown |
| append                   | (...) if data/table already exists, contents of the **DataFrame** are expected to be appended to existing data |
| overwrite                | (...) if data/table already exists, existing data is expected to be overwritten by the contents of the **DataFrame** |
| ignore                   | (...) if data already exists, the save operation is expected to not save the contents of the DataFrame and to not change the existing data |

### The DataFrames/Datasets API

Creating dataframe

In [12]:
from pyspark.sql import Row
from pyspark.sql.types import IntegerType

r = [Row(identification=1,name='John',surname='Doe',age=29),
     Row(identification=2,name='Jane',surname='Doe',age=25)]
df_data = spark.createDataFrame(r)

df_data.show()

+---+--------------+----+-------+
|age|identification|name|surname|
+---+--------------+----+-------+
| 29|             1|John|    Doe|
| 25|             2|Jane|    Doe|
+---+--------------+----+-------+



### SparkSQL

### Caching and caching storage levels

---

### References

1. Kumar, R., 2018. ***PySpark Recipes***. Apress.
2. Tomasz, D., 2017. ***Learning PySpark***. Packt Publishing.
3. Spark.apache.org. (2018). ***Apache Spark™ - Unified Analytics Engine for Big Data***. [online] Available at: http://spark.apache.org/ [Accessed 25 Sep. 2018].
4. Spark.apache.org. (2018). ***GraphX | Apache Spark***. [online] Available at: http://spark.apache.org/graphx/ [Accessed 25 Sep. 2018].
5. Spark.apache.org. (2018). ***Spark Streaming - Spark 2.3.1 Documentation***. [online] Available at: https://spark.apache.org/docs/latest/streaming-programming-guide.html [Accessed 25 Sep. 2018].
6. DeZyre. (2018). **Apache Spark Architecture Explained in Detail**. [online] Available at: https://www.dezyre.com/article/apache-spark-architecture-explained-in-detail/338 [Accessed 26 Sep. 2018].
7. Data-flair.training. (2018). Apache Spark Ecosystem – Complete Spark Components Guide – DataFlair. [online] Available at: https://data-flair.training/blogs/apache-spark-ecosystem-components/ [Accessed 1 Oct. 2018].